In [ ]:
import sqlite3
import langchain
import langchain_community
from langchain_community.utilities import SQLDatabase

In [ ]:
db_path = r"./POC-LangChain/chinook-database-master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [2]:
from langchain.chains import LLMChain
from typing_extensions import TypedDict
import re

# Define your State class
class State(TypedDict):
    schema_info: str       # Loaded information about the DB schema
    question: str          # Original natural language question
    db_dialect: str        # SQL dialect (e.g., "sqlite")
    parsed_question: str   # (Optional) Parsed or analyzed question
    query: str             # Generated SQL query
    execution_result: str  # Result from executing the SQL query
    answer: str            # Final answer formatted for the user
    error: str             # Error message (if any)

# Function to extract SQL statement from text
def extract_sql_statement(text: str) -> str:
    """
    Extracts the SQL statement from a given text, starting from "SELECT" 
    until the first semicolon ";".
    """
    pattern = r'(SELECT.*?;)'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

# New helper function that uses chain.invoke and updates the State
def extract_sql_query_state(prompt_template, state: State) -> State:
    """
    Uses an LLMChain to generate an SQL query from the natural language question 
    contained in the provided state. Updates the state's 'query' field with the 
    extracted SQL and sets an 'error' field if exceptions occur.
    """
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    try:
        # Call chain.invoke with a dictionary as input
        output = chain.invoke({
            "schema_info": state["schema_info"],
            "question": state["question"],
            "db_dialect": state["db_dialect"]
        })
        
        # Optionally extract the SQL statement (if extra formatting is added)
        sql_query = extract_sql_statement(output)
        state["query"] = sql_query if sql_query else output
        state["error"] = ""
    except Exception as e:
        state["error"] = str(e)
    
    return state

# Example usage:

# Assuming you have created the prompt_template and have the following variables defined:
# schema_info_str, question, and db_dialect

# Create an initial state
state: State = {
    "schema_info": schema_info_str,
    "question": question,
    "db_dialect": db_dialect,
    "parsed_question": "",
    "query": "",
    "execution_result": "",
    "answer": "",
    "error": ""
}

# Invoke the chain to update the state with the generated SQL query
state = extract_sql_query_state(prompt_template, state)

print("Generated SQL Query:")
print(state["query"])
if state["error"]:
    print("Error encountered:", state["error"])


NameError: name 'schema_info_str' is not defined